In [15]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
import math
from sklearn.model_selection import GridSearchCV
import subprocess
from sklearn.utils import shuffle
import itertools
import joblib
import copy


In [14]:
def read_fasta_file(path):
    '''
    used for load fasta data and transformd into numpy.array format
    '''
    fh=open(path)
    seq=[]
    for line in fh:
        if line.startswith('>'):
            continue
        else:
            seq.append(line.replace('\r\n',''))
    fh.close()
    matrix_data=np.array([list(e) for e in seq])
    return matrix_data

In [47]:
def AthMethPre_extract_one_line(data_line,gram):
    '''
    extract features from one line, such as one m6A sample
    '''
    alphabet='ACGU'
    matrix_two=["".join(e) for e in itertools.product(alphabet, repeat=gram)] # AA AU AC AG UU UC ...
    matrix_one=["".join(e) for e in itertools.product(alphabet,repeat=gram-1)]
    Nab=np.zeros(int(pow(4,gram)))
    NaA=np.zeros(int(pow(4,gram-1)))
    for index,data in enumerate(data_line):
        if "".join(data_line[index:index+gram]) in matrix_two and index<=len(data_line)-gram:
                 Nab[matrix_two.index("".join(data_line[index:index+gram]))]=Nab[matrix_two.index("".join(data_line[index:index+gram]))]+1
        if "".join(data_line[index:index+gram-1]) in matrix_one and index<=len(data_line)-gram:
                 NaA[matrix_one.index("".join(data_line[index:index+gram-1]))]=NaA[matrix_one.index("".join(data_line[index:index+gram-1]))]+1
    Nab2=copy.deepcopy(Nab)
    for index,data in enumerate(matrix_two):
        if Nab[index]!=0:
            if data[0:gram-1] in matrix_one:
                 Nab[index]=Nab[index]/NaA[matrix_one.index(data[0:gram-1])]
    Nab2=Nab2/(len(data_line)-1)
    one_line_result=[]
    one_line_result.extend(Nab)
    one_line_result.extend(Nab2)
    return one_line_result

In [10]:
import numpy as np
a=np.array([1,2,3])
b=np.array([4,5,6])
lis=[]
lis.extend(a)
lis.extend(b)
print lis

[1, 2, 3, 4, 5, 6]


In [45]:
def AthMethPre_feature_extraction(matrix_data,gram):
    final_feature_matrix=[AthMethPre_extract_one_line(e,gram) for e in matrix_data]
    print np.array(final_feature_matrix).shape
    return final_feature_matrix

In [27]:


def generating_csv_file(gram):
    m6a_benchmark_dataset='/home02/chenhuangrong/m6a_data.txt'
    matrix_data=read_fasta_file(m6a_benchmark_dataset)
    final_feature_matrix=AthMethPre_feature_extraction(matrix_data,gram)
    print np.array(final_feature_matrix).shape
    pd.DataFrame(final_feature_matrix).to_csv('/home02/chenhuangrong/%dNC.csv'%(gram),header=None,index=False)




In [19]:
def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError

In [20]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [70]:
"""
used to merge features
"""
RFH_=pd.read_csv("/home02/chenhuangrong/PSNP.csv",header=None,index_col=None)#sys.argv[1])
RFH_=RFH_.values
RFH_=pd.DataFrame(RFH_).astype(float)
PseDNC_=pd.read_csv("/home02/chenhuangrong/PSDNP_1_gap.csv",header=None,index_col=None)
PseDNC_=pd.DataFrame(PseDNC_).astype(float)
print "first_file_num:",len(RFH_)
print "first_file_length:",len(RFH_.values[0])
print "second_file_num:",len(PseDNC_)
print "second_file_length:",len(PseDNC_.values[0])
RFH_PseDNC=pd.concat([RFH_,PseDNC_],axis=1)
print "output_file_num:",len(RFH_PseDNC)
print "output_file_len:",len(RFH_PseDNC.values[0])
scaler=MinMaxScaler()
RFH_PseDNC=scaler.fit_transform(np.array(RFH_PseDNC))

first_file_num: 2614
first_file_length: 3
second_file_num: 2614
second_file_length: 3
output_file_num: 2614
output_file_len: 6


In [36]:
"""
    cross validation
"""
def generating_crossvalidation(gram):
    print gram
    datapath ='/home02/chenhuangrong/%dNC.csv'%(gram)
    train_data = pd.read_csv(datapath, header=None, index_col=None)
    X = np.array(train_data)
    Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
    Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
    Y.extend(Y2)
    Y = np.array(Y)
    svc = svm.SVC()
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=-1, scoring='accuracy')
    clf.fit(X, Y)
    C=clf.best_params_['C']
    # joblib.dump(clf,'/home02/chenhuangrong/%d_gap_%d_gram.csv'%(gap,gram))
    print clf.best_score_
    gamma=clf.best_params_['gamma']
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,),X,Y,cv=10,n_jobs=-1)
    print type(y_predict)
    y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=-1,method='predict_proba')
    predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    predict_save=np.array(predict_save).T
    pd.DataFrame(predict_save).to_csv('/home02/chenhuangrong/blending_files/%d_NC.csv'%(gram),header=None,index=False)
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y,y_predict)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
    print savedata
#     easy_excel.save("svm_crossvalidation",[str(X.shape[1])],savedata,'/home02/chenhuangrong/cross_validation_frequency_%d_gram.xls'%(gram))
    return ACC
    # y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
    # Y=pd.DataFrame(Y)    
    # y_predict_proba=pd.DataFrame(y_predict_proba)
    # y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
    # pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
    # y_predict=pd.DataFrame(y_predict)
    # y_predict_=pd.concat([Y,y_predict],axis=1)
    # pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

In [ ]:

ACC_=0
for gram in xrange(2,9):
    generating_csv_file(gram)
    ACC=generating_crossvalidation(gram)
    if ACC_<ACC:
        ACC_=ACC
print ACC_

(2614, 32)
(2614, 32)
2
0.644605967865
<type 'numpy.ndarray'>
[[['svmC:32.0gamma:0.157490131237', 0.64460596786534052, 0.6308864265927978, 0.6970160673297628, 0.6970160673297628, 0.5921958684009181, 0.6424718167217467, 0.66230461650308992, 0.66230461650308992, 0.29081397482285709, '0.6445', 911.0, 396.0, 533.0, 774.0, 1307.0, 1307.0]]]
(2614, 128)
(2614, 128)
3


In [14]:
a=np.array([1,2,3])
b=np.array([4,5,6])
li=[a,b]
print np.array(li).T

[[1 4]
 [2 5]
 [3 6]]


In [103]:
final_out_to_excel=[]
row0 = [u'特征集', u'样本个数', u'分类器', u'Accuracy', u'Precision', u'Recall', u'SN', u'SP',
                u'Gm', u'F_measure', u'F_score', u'MCC', u'ROC曲线面积', u'tp', u'fn', u'fp', u'tn']
final_out_to_excel.append(row0) #above was used to generate xlsx format Excel file
RNA_code='ACGU' 
interval=3 # RNA_code and interval used mix used to generate AAA AAC AAG ...
division_num=10
divided_num=np.float(division_num)# ten fold so the result should divided by ten


# seq=[e.replace('U','A') for e in seq]
# seq=[e.replace('G','C') for e in seq]
seq=pd.read_csv('/home02/chenhuangrong/%d_gap_%d_gram.csv'%(gap,gram),header=None,index_col=False)
seq=seq.values

positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]



#define variable to save data which will be saved later --- begining
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0
#define variable to save data which will be saved later --- end


#shuffle the data of positive and negative begining
kf = KFold(n_splits=division_num,shuffle=True)  # positive and negative samples will be shuffled 
for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
#shuffle the data of positive and negative end


    #generate train and test data begining
    X_train = np.array(positive_negative_x_train)
    Y_train = list(map(lambda x: 1, xrange(len(X_train) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(X_train) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(positive_negative_y_train)
    Y_test  = list(map(lambda x: 1, xrange(len(X_test) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(X_test) / 2)))
    Y_test.extend(Y2_test )
    Y_test = np.array(Y_test)
    #generate train and test data end

    
    # training model and optimized parameters of C and gamma begining
    svc = svm.SVC(probability=True)
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    # training model and optimized parameters of C and gamma end
    
    
    #print best C and gamma begining
    C=clf.best_params_['C']
    gamma=clf.best_params_['gamma']
    print 'c:',C,'gamma:',gamma
    #print best C and gamma end
    
    
    #getting predict probability and predict label begining
    y_pred_prob=clf.predict_proba(X_test)
    y_pred=clf.predict(X_test)
    #getting predict probability and predict label begining

    #the process of generating  usefule data begining
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)

    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
    #the process of generating  usefule data end
    
#the process of save  data to disk begining   
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
# pd.DataFrame(np.matrix(all_y).T).to_csv('/home02/chenhuangrong/PseEIIP%d.csv'%select_num,header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[str(X_train.shape[1]),"正："+str(pos_all)+'负：'+str(neg_all),'svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all]
final_out_to_excel.append(savedata)

print savedata
pd.DataFrame(final_out_to_excel).to_excel('/home02/chenhuangrong/%d_gap_%d_gram.xlsx'%(gap,gram),sheet_name="independent_test",index=False,header=False)
#the process of save  data to disk end

c: 0.25 gamma: 0.157490131237
0.931297709924
ACC_all: 0.931297709924
c: 0.25 gamma: 0.0701538780193
0.935114503817
ACC_all: 1.86641221374
c: 0.25 gamma: 0.0701538780193
0.912213740458
ACC_all: 2.7786259542
c: 0.25 gamma: 0.157490131237
0.958015267176
ACC_all: 3.73664122137
c: 0.25 gamma: 0.157490131237
0.950381679389
ACC_all: 4.68702290076
c: 0.25 gamma: 0.157490131237
0.923664122137
ACC_all: 5.6106870229
c: 0.25 gamma: 4.0
0.954198473282
ACC_all: 6.56488549618
c: 0.25 gamma: 0.157490131237
0.942307692308
ACC_all: 7.50719318849
c: 0.25 gamma: 0.157490131237
0.938461538462
ACC_all: 8.44565472695
c: 0.25 gamma: 0.0701538780193
0.946153846154
ACC_all: 9.39180857311
['64', '\xe6\xad\xa3\xef\xbc\x9a1307.0\xe8\xb4\x9f\xef\xbc\x9a1307.0', 'svmC:0.25gamma:0.0701538780193', 0.93918085731062817, 0.9799303685180801, 0.8967116852613037, 0.8967116852613037, 0.981650029359953, 0.9381809940519183, 0.9364019521621284, 0.9364019521621284, 0.88164235079910092, 0.49011795118861479, 1172.0, 135.0, 24.0, 1

In [122]:
"""
used to test the final performance with other dataset
"""

m6a_benchmark_dataset='/home02/chenhuangrong/Homo_sapiens_used_test.fasta'
matrix_data=read_fasta_file(m6a_benchmark_dataset)
final_feature_matrix=AthMethPre_feature_extraction(matrix_data)
print np.array(final_feature_matrix).shape
final_feature_matrix=pd.DataFrame(final_feature_matrix)
print final_feature_matrix
Y_train = list(map(lambda x: 1, xrange(len(final_feature_matrix.values) / 2)))
Y2_train = list(map(lambda x: 0, xrange(len(final_feature_matrix.values) / 2)))
Y_train.extend(Y2_train)
print len(Y_train)
clf=joblib.load('/home02/chenhuangrong/%d_gap_%d_gram.csv'%(gap,gram))
#getting predict probability and predict label begining
y_pred=clf.predict(final_feature_matrix.values)
#getting predict probability and predict label begining


ACC=metrics.accuracy_score(Y_train,y_pred)
print ACC
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_train, y_pred) 
F1_Score=metrics.f1_score(Y_train, y_pred)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y_train, y_pred)
print MCC

(16732, 64)
             0         1         2         3         4         5         6   \
0      0.020738  0.018946  0.016137  0.018228  0.016256  0.013387  0.012013   
1      0.020738  0.018945  0.016136  0.018228  0.016256  0.013387  0.012013   
2      0.020738  0.018945  0.016136  0.018228  0.016256  0.013387  0.012012   
3      0.020738  0.018945  0.016136  0.018228  0.016255  0.013387  0.012012   
4      0.020737  0.018945  0.016136  0.018227  0.016255  0.013387  0.012012   
5      0.020737  0.018944  0.016136  0.018227  0.016255  0.013387  0.012012   
6      0.020737  0.018944  0.016135  0.018227  0.016255  0.013386  0.012012   
7      0.020737  0.018944  0.016135  0.018227  0.016255  0.013386  0.012012   
8      0.020736  0.018944  0.016135  0.018227  0.016255  0.013386  0.012012   
9      0.020736  0.018943  0.016135  0.018226  0.016254  0.013386  0.012011   
10     0.020736  0.018943  0.016135  0.018226  0.016254  0.013398  0.012011   
11     0.020736  0.018943  0.016134  0.0